In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from Vis import ClassificationVis, LabelVis
from Evals import *
from Data_Provider import *
import util.classification_utils as util
import util.data_utils as dutil
import util.label_utils as lutil

import numpy as np
from scipy import interp
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier


from itertools import product
from multiprocessing import Pool

In [ ]:
#Run classifier with cross-validation and plot ROC curves
def calc_results_and_save(x,y,configs, shuff):
    cv = StratifiedKFold(n_splits=10,shuffle=shuff,random_state = 1)
    results = pd.DataFrame(columns=('C','Gamma','AVG PR'))#,'AUC ROC'))
    cs = np.random.uniform(0,5,50)
    gammas = 2**(np.random.choice(18,50)-15.)

    for idx,(g,c) in enumerate(zip(gammas,cs)):
        classifier = svm.SVC(cache_size=16000,C=c,kernel='rbf', probability = True,gamma = g, random_state=5)
        auc_pr = util.get_auc_score(classifier, cv, x, y, go_after_pr=True)
        #auc_roc =util.get_auc_score(classifier, cv, x, y, go_after_pr=False)
        results.loc[idx] = [c,g,auc_pr]
        print('Gamma = %.6f, C= %2.2f, Avg PR: %.4f' % (g,c,auc_pr))
    if shuff:
        dutil.save_results(results,configs,'RBF') # save results for later use
    else:
        dutil.save_results(results,configs,'RBF_NoShuffle') # save results for later use
    return results


def vis_results(x,y, x_ev, y_ev, hypers,shuff):
    best_c, best_g = hypers
    ###using hyperpara found, evaluate and get pretty plots
    classifier = svm.SVC(C=best_c, kernel='rbf', probability=True,gamma = best_g,random_state=5)
    cv = StratifiedKFold(n_splits=10, shuffle=shuff, random_state=1)
    best_thr = util.get_optimal_threshold(classifier, cv, x, y, go_after_pr=True) # get threshold using cv
    #for thr in np.linspace(0,1,10):
    print(best_thr)
    y_pred,y_pred_ev = util.fit_predict(classifier, best_thr, x, y, x_ev, y_ev) # using that threshold, get predictions and f1 score
    f1_tr=get_f1(y_pred,y) # calculate f1 scores for prediction on train set
    f1_ev=get_f1(y_pred_ev,y_ev)
    prec_tr,recall_tr = get_precision_recall(y_pred,y)
    prec_ev,recall_ev = get_precision_recall(y_pred_ev,y_ev)

    print(prec_tr, recall_tr,f1_tr)
    print(prec_ev, recall_ev,f1_ev)

    
    ClassificationVis.conf_mat(y_pred,y)
    ClassificationVis.conf_mat(y_pred_ev,y_ev)
    plt.show()
    classifier.fit(x,y)
    ClassificationVis.plot_roc(x,y,classifier, 'SVC, RBF Kernel, C ={:.2f}, Gamma = {:.5f}'.format(best_c,best_g))
    ClassificationVis.plot_roc(x_ev,y_ev,classifier, 'SVC, RBF Kernel, C ={:.2f}, Gamma = {:.5f}'.format(best_c,best_g))
    ClassificationVis.plot_pr_curve(x,y,classifier,'SVC, RBF Kernel, C ={:.2f}, Gamma = {:.5f}'.format(best_c,best_g))
    ClassificationVis.plot_pr_curve(x_ev,y_ev,classifier,'SVC, RBF Kernel, C ={:.2f}, Gamma = {:.5f}'.format(best_c,best_g))

def do_all(file, cut,shuff):
    provider = DataProvider()
    configs = dutil.generate_configs_from_file(file, cut)
    print(configs)
    x,y,x_ev,y_ev = provider.get_data(configs)
    print(x.shape,y.shape,x_ev.shape,y_ev.shape)
    res = calc_results_and_save(x, y, configs, shuff)
    hypers = util.get_best_hyperparas(res)
    vis_results(x,y, x_ev, y_ev,hypers)

In [ ]:
%matplotlib inline

In [ ]:
files = [f for f in os.listdir('/home/emil/OpenMindv2/data/several_days') if f.endswith('hdf')]
print(files)
shuff=False
#cuts = [.1,.2,.4]
cuts = [.1,.2,.3]
all_elements = [files,cuts,[shuff]]
file_cut_combos = []
for allel in product(*all_elements):
    file_cut_combos+=[allel]
print(file_cut_combos)    
pool = mp.Pool(8)
#yass = pool.starmap(self.load_raws_single_day,list(zip([patient]*len(days),days)))
yass = pool.starmap(do_all,file_cut_combos)